In [5]:
import requests

url = 'https://jobs.github.com/positions.json'
search_term = 'data science'

params = {'description': search_term}
response = requests.get(url, params=params)

print(response.status_code)

200


In [9]:
json_response = response.json()

print('There are {n} results for the term "{term}".'
      .format(n=len(json_response), term=search_term))

There are 27 results for the term "data science".


In [7]:
import json
print(json_response == json.loads(response.text))

True


In [11]:
print(json.dumps(json_response, sort_keys=True, indent=4))

[
    {
        "company": "Network Merchants, LLC",
        "company_logo": "http://github-jobs.s3.amazonaws.com/a1b8f9ec-bbed-11e6-960a-b1034c4cd71b.png",
        "company_url": "http://nmi.com",
        "created_at": "Tue Dec 06 19:54:25 UTC 2016",
        "description": "<p><strong>Summary</strong> </p>\n\n<ul>\n<li>The engineering team at NMI is a small, agile group that works closely together but requires independent thinking and motivation. Engineers have direct influence over the design and implementation of systems, and infrastructure. Expertise should include fluency in Linux and security, strong troubleshooting and analytical skills, good communication skills and a genuine passion for solving problems related to scalability, performance, security and redundancy.</li>\n</ul>\n\n<p><strong>Essential Functions</strong> </p>\n\n<ul>\n<li>Communicate with staff or clients to understand specific system requirements.</li>\n<li>Provide advice on project costs, design concepts, or de

In [12]:
first_job = json_response.pop()
first_job.keys()

dict_keys(['id', 'how_to_apply', 'description', 'location', 'company_url', 'title', 'company_logo', 'type', 'url', 'created_at', 'company'])

In [13]:
from IPython.display import Image
image_url = first_job['company_logo']
Image(url=image_url) 

In [14]:
print(first_job['type'], 
      first_job['company'], 
      first_job['company_url'],
      first_job['title'],
      first_job['location'],
      first_job['created_at'],
      sep='\n')

Full Time
Kaggle
http://www.kaggle.com
Full-Stack Engineer (C#, ASP.NET MVC and React)
San Francisco or Remote
Mon Nov 07 08:42:27 UTC 2016


In [16]:
from IPython.core.display import display, HTML

display(HTML(first_job['description']))

In [17]:
display(HTML(first_job['how_to_apply']))

In [18]:
print(first_job['id'], 
      first_job['url'],
      sep='\n')

a0389ac2-39df-11e6-9bb0-7b3faa3729cc
http://jobs.github.com/positions/a0389ac2-39df-11e6-9bb0-7b3faa3729cc


In [20]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey, DateTime

engine = create_engine('sqlite:///:memory:', echo=True)
metadata = MetaData()

company_table = Table('company', metadata,
                      Column('id', Integer, primary_key=True),
                      Column('name', String, nullable=False),
                      Column('url', String, unique=True),
                      Column('logo_url', String))

job_table = Table('job', metadata,
                  Column('id', Integer, primary_key=True),
                  Column('company_id', None, ForeignKey('company.id'), nullable=False),
                  Column('type_', Integer),
                  Column('title', String, nullable=False),
                  Column('description', String),
                  Column('how_to_apply', String))

location_table = Table('location', metadata,
                      Column('job_id', None, ForeignKey('job.id'), primary_key=True),
                      Column('description', String),
                      Column('remote_ok', Integer))


github_metadata_table = Table('github_metadata', metadata,
                              Column('job_id', None, ForeignKey('job.id'), primary_key=True),
                              Column('github_id', String, unique=True),
                              Column('url', String, nullable=False),
                              Column('created_at', DateTime, nullable=False))

metadata.create_all(engine)
conn = engine.connect()

2016-12-07 23:32:24,804 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-12-07 23:32:24,807 INFO sqlalchemy.engine.base.Engine ()
2016-12-07 23:32:24,809 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-12-07 23:32:24,811 INFO sqlalchemy.engine.base.Engine ()
2016-12-07 23:32:24,813 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("location")
2016-12-07 23:32:24,814 INFO sqlalchemy.engine.base.Engine ()
2016-12-07 23:32:24,816 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("job")
2016-12-07 23:32:24,820 INFO sqlalchemy.engine.base.Engine ()
2016-12-07 23:32:24,822 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("github_metadata")
2016-12-07 23:32:24,831 INFO sqlalchemy.engine.base.Engine ()
2016-12-07 23:32:24,833 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("company")
2016-12-07 23:32:24,837 INFO sqlalchemy.engine.base.Engine ()
2016-12-07 23:32:24,840 INFO sqla

In [28]:
from enum import Enum

class JobType(Enum):
    full_time = 1
    part_time = 2
    
    @staticmethod
    def from_string(string):
        if string.lower() == 'full time' or string.lower() == 'full-time':
            return JobType.full_time
        elif string.lower() == 'part time' or string.lower() == 'part-time':
            return JobType.part_time
        else:
            raise RuntimeError('JobType not supported: {}'.format(string))

In [30]:
def insert_company(name, url, logo_url, connection=conn):
    """Inserts a new entry in company_table and returns its id."""
    insert = company_table.insert().values(name=name, url=url, logo_url=logo_url)
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

In [31]:
def insert_location(job_id, description, connection=conn):
    """Inserts a new entry in location_table and returns its job_id."""
    insert = location_table.insert().values(job_id=job_id, description=description, 
                                       remote_ok=int('remote' in description.lower() 
                                                     or 'telecommute' in description.lower())) 
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

In [32]:
import datetime
created_at_pattern = '%a %b %d %X %Z %Y'

def insert_github_metadata(job_id, github_id, url, created_at_str, connection=conn):
    """Inserts a new entry in github_metadata_table and returns its job_id."""
    insert = github_metadata_table.insert().values(job_id=job_id, github_id=github_id, url=url,
                                       created_at=datetime.datetime.strptime(created_at_str, created_at_pattern))
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

In [38]:
def insert_job(company_id, type_str, title, description, how_to_apply, connection=conn):
    """Inserts a new entry in job_table and returns its id."""
    insert = job_table.insert().values(company_id=company_id, 
                                       type_=JobType.from_string(type_str).value, 
                                       title=title, description=description, how_to_apply=how_to_apply) 
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

int

In [34]:
from sqlalchemy.sql import select

def select_or_insert_company(name, url, logo_url, connection=conn):
    """Returns the company id either by selecting from or inserting in company_table"""
    s = select([company_table]).where(company_table.c.url == url)
    result = connection.execute(s).fetchone()
    if (result is not None):
        return result['id']
    else:
        new_id = insert_company(name, url, logo_url, connection)
        return new_id

In [35]:
def select_or_insert_job(company_name, company_url, company_logo_url, 
                         github_id, github_url, github_created_at_str,
                         type_str, title, description, how_to_apply,
                         location_description):
    """
    Returns the job id either by selecting from or inserting in job_table.
    
    This function might also insert new location, github_metadata and company entries.
    """
    s = select([github_metadata_table]).where(github_metadata_table.c.github_id == github_id)
    result = conn.execute(s).fetchone()
    if (result is not None):
        return result['job_id']
    else:
        with engine.begin() as transaction:
            company_id = select_or_insert_company(company_name, company_url, company_logo_url, transaction)
            job_id = insert_job(company_id, type_str, title, description, how_to_apply, transaction)
            insert_github_metadata(job_id, github_id, github_url, github_created_at_str, transaction)
            insert_location(job_id, location_description, transaction)
            
        return job_id

In [39]:
for job in json_response:
    select_or_insert_job(
        job['company'],
        job['company_url'],
        job['company_logo'],
        job['id'],
        job['url'],
        job['created_at'],
        job['type'],
        job['title'],
        job['description'],
        job['how_to_apply'],
        job['location']
    )

2016-12-08 00:02:09,083 INFO sqlalchemy.engine.base.Engine SELECT github_metadata.job_id, github_metadata.github_id, github_metadata.url, github_metadata.created_at 
FROM github_metadata 
WHERE github_metadata.github_id = ?
2016-12-08 00:02:09,086 INFO sqlalchemy.engine.base.Engine ('beb74684-bbed-11e6-8197-64e617b51adf',)
2016-12-08 00:02:09,088 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-08 00:02:09,090 INFO sqlalchemy.engine.base.Engine SELECT company.id, company.name, company.url, company.logo_url 
FROM company 
WHERE company.url = ?
2016-12-08 00:02:09,091 INFO sqlalchemy.engine.base.Engine ('http://nmi.com',)
2016-12-08 00:02:09,092 INFO sqlalchemy.engine.base.Engine INSERT INTO company (name, url, logo_url) VALUES (?, ?, ?)
2016-12-08 00:02:09,094 INFO sqlalchemy.engine.base.Engine ('Network Merchants, LLC', 'http://nmi.com', 'http://github-jobs.s3.amazonaws.com/a1b8f9ec-bbed-11e6-960a-b1034c4cd71b.png')
2016-12-08 00:02:09,098 INFO sqlalchemy.engine.base.Engine 

In [40]:
from sqlalchemy.sql import text

s = text('SELECT * FROM company')
result = conn.execute(s)

for row in result:
    print('\n {}'.format(row))

2016-12-08 00:04:57,202 INFO sqlalchemy.engine.base.Engine SELECT * FROM company
2016-12-08 00:04:57,204 INFO sqlalchemy.engine.base.Engine ()

 (1, 'Network Merchants, LLC', 'http://nmi.com', 'http://github-jobs.s3.amazonaws.com/a1b8f9ec-bbed-11e6-960a-b1034c4cd71b.png')

 (2, 'Stevens Capital Management LP', 'http://www.scm-lp.com', 'http://github-jobs.s3.amazonaws.com/5df90c6a-baf8-11e6-8a38-492033c01d86.png')

 (3, 'GenePeeks, Inc', 'http://www.genepeeks.com', 'http://github-jobs.s3.amazonaws.com/67ea79ea-b8c6-11e6-81ec-ba6eeed777c6.jpg')

 (4, 'Intel Corporation', 'http://jobs.intel.com', 'http://github-jobs.s3.amazonaws.com/0590279e-b7eb-11e6-92e0-095053d3f49d.png')

 (5, 'PDT Partners', 'http://www.pdtpartners.com', 'http://github-jobs.s3.amazonaws.com/1dee7540-b7f8-11e6-8780-56e37987ae3b.png')

 (6, 'CrowdFlower', None, None)

 (7, 'Carbon Black, Inc.', 'https://www.carbonblack.com/', 'http://github-jobs.s3.amazonaws.com/309b531e-b679-11e6-8bed-4a8a3761e6da.jpg')

 (8, 'Smarter

In [41]:
s = text('SELECT * FROM job')
result = conn.execute(s)

for row in result:
    print('\n {}'.format(row))

2016-12-08 00:06:50,892 INFO sqlalchemy.engine.base.Engine SELECT * FROM job
2016-12-08 00:06:50,894 INFO sqlalchemy.engine.base.Engine ()

 (1, 1, 1, 'Linux Systems Engineer', '<p><strong>Summary</strong> </p>\n\n<ul>\n<li>The engineering team at NMI is a small, agile group that works closely together but requires independen ... (2957 characters truncated) ... >\n<li>Jira and Confluence for task and project management</li>\n<li>Computer</li>\n<li>Printer</li>\n<li>Fax Machine</li>\n<li>Scanner</li>\n</ul>\n', '<p>Please E-mail your resume to <a href="mailto:hr@nmi.com">hr@nmi.com</a></p>\n')

 (2, 2, 1, 'Implementation Developer', '<p>Stevens Capital Management LP (“SCM”) is a registered investment adviser that manages a multi-billion dollar hedge fund that has been in business  ... (1396 characters truncated) ... Experience with relational databases including Sybase, SQL Server and Oracle is a plus.</li>\n<li>Experience with GUI design is a plus.</li>\n</ul>\n', '<p>Please submit you

In [42]:
s = text('SELECT * FROM location')
result = conn.execute(s)

for row in result:
    print('\n {}'.format(row))

2016-12-08 00:07:38,221 INFO sqlalchemy.engine.base.Engine SELECT * FROM location
2016-12-08 00:07:38,225 INFO sqlalchemy.engine.base.Engine ()

 (1, 'Roselle, IL', 0)

 (2, 'Radnor, PA. USA', 0)

 (3, 'Cambridge, MA', 0)

 (4, 'Karlsruhe, Germany', 0)

 (5, 'Gdansk, Poland ', 0)

 (6, 'New York', 0)

 (7, 'San Francisco, CA, US', 0)

 (8, 'Waltham, MA or Telecommute', 1)

 (9, 'New York, NY', 0)

 (10, 'Stockholm', 0)

 (11, 'Amsterdam', 0)

 (12, 'Amsterdam', 0)

 (13, 'Seattle', 0)

 (14, 'Redwood City, CA', 0)

 (15, 'New York, NY', 0)

 (16, 'Sierra Leone', 0)

 (17, 'Sierra Leone', 0)

 (18, 'Irvine, CA', 0)

 (19, 'Palo Alto', 0)

 (20, 'Minneapolis, MN', 0)

 (21, 'New York, NY', 0)

 (22, 'San Francisco, CA', 0)

 (23, 'San Francisco, CA', 0)

 (24, 'Waterloo, Canada', 0)

 (25, 'Amsterdam', 0)

 (26, 'Amsterdam', 0)


In [43]:
s = text('SELECT * FROM github_metadata')
result = conn.execute(s)

for row in result:
    print('\n {}'.format(row))

2016-12-08 00:08:35,161 INFO sqlalchemy.engine.base.Engine SELECT * FROM github_metadata
2016-12-08 00:08:35,166 INFO sqlalchemy.engine.base.Engine ()

 (1, 'beb74684-bbed-11e6-8197-64e617b51adf', 'http://jobs.github.com/positions/beb74684-bbed-11e6-8197-64e617b51adf', '2016-12-06 19:54:25.000000')

 (2, '61433012-baf8-11e6-8d18-1baa36e6dbf5', 'http://jobs.github.com/positions/61433012-baf8-11e6-8d18-1baa36e6dbf5', '2016-12-05 14:38:11.000000')

 (3, '6d409aaa-b8c6-11e6-8408-7debc09d256f', 'http://jobs.github.com/positions/6d409aaa-b8c6-11e6-8408-7debc09d256f', '2016-12-02 19:36:23.000000')

 (4, '47b78b5e-b7ef-11e6-86f0-a00baabc0c92', 'http://jobs.github.com/positions/47b78b5e-b7ef-11e6-86f0-a00baabc0c92', '2016-12-02 11:49:17.000000')

 (5, '88d7a11e-b7ef-11e6-8839-4baedb4f1c00', 'http://jobs.github.com/positions/88d7a11e-b7ef-11e6-8839-4baedb4f1c00', '2016-12-02 11:49:15.000000')

 (6, '24d6d212-b7f8-11e6-9bc0-4c468fa010c5', 'http://jobs.github.com/positions/24d6d212-b7f8-11e6-9bc0-

# This should be the output from our queries:

There are 27 jobs with the 'Data Science' term, and 40% of them have 'Python' in the description.

22% of them have been created in the past 7 days.

2 of those are remote ok. The companies with remote jobs are: ['Carbon Black, Inc.', 'Kaggle']

In [49]:
from sqlalchemy import func

s = text("SELECT COUNT() "
         "FROM job")
total, = conn.execute(s).fetchone()

# s = select([func.count()]).select_from(job_table).where(
#     job_table.c.description.contains('python'))
s = text("SELECT COUNT() "
         "FROM job "
         "WHERE description LIKE '%python%'")
python, = conn.execute(s).fetchone()

print("\nThere are {} jobs with the 'Data Science' term, and {}% of them have 'Python' in the description."
      .format(total, 100 * python // total))

2016-12-08 00:16:35,572 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM job
2016-12-08 00:16:35,573 INFO sqlalchemy.engine.base.Engine ()
2016-12-08 00:16:35,576 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM job WHERE description LIKE '%python%'
2016-12-08 00:16:35,577 INFO sqlalchemy.engine.base.Engine ()

There are 26 jobs with the 'Data Science' term, and 38% of them have 'Python' in the description.


In [52]:
from datetime import datetime, timedelta

s = text("SELECT COUNT() "
         "FROM github_metadata "
         "WHERE created_at >= :week_ago")
last_week, = conn.execute(s, week_ago=datetime.today() - timedelta(days=7)
                         ).fetchone()

print("\n{}% of them have been created in the past 7 days."
      .format(100 * last_week // total))

2016-12-08 00:19:56,993 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM github_metadata WHERE created_at >= ?
2016-12-08 00:19:56,995 INFO sqlalchemy.engine.base.Engine (datetime.datetime(2016, 12, 1, 0, 19, 56, 993263),)

23% of them have been created in the past 7 days.


In [55]:
s = text("SELECT COUNT() "
         "FROM job "
         "JOIN location ON job.id = location.job_id "
         "WHERE remote_ok = :true")
remote_jobs, = conn.execute(s, true=int(True)).fetchone()

# s = select([job_table.c.title]).select_from(job_table.join(location_table)).where(
#      location_table.c.remote_ok == int(True))
s = text("SELECT name "
         "FROM company "
         "JOIN job ON company.id = job.company_id "
         "JOIN location ON job.id = location.job_id "
         "WHERE location.remote_ok = :true")
remote_companies = conn.execute(s, true=int(True)).fetchall()

print("\n{} of those is remote ok. The company with the remote job is: {}"
      .format(remote_jobs, [company for company, in remote_companies]))

2016-12-08 00:25:29,613 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM job JOIN location ON job.id = location.job_id WHERE remote_ok = ?
2016-12-08 00:25:29,615 INFO sqlalchemy.engine.base.Engine (1,)
2016-12-08 00:25:29,616 INFO sqlalchemy.engine.base.Engine SELECT name FROM company JOIN job ON company.id = job.company_id JOIN location ON job.id = location.job_id WHERE location.remote_ok = ?
2016-12-08 00:25:29,617 INFO sqlalchemy.engine.base.Engine (1,)

1 of those is remote ok. The company with the remote job is: ['Carbon Black, Inc.']


In [56]:
conn.close()